<a href="https://colab.research.google.com/github/natdebandi/hate_speech_ar/blob/main/1_finetunning_BETO_bin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP final - reconocimiento de discursos discriminatorios en Twitter
## (1) Finetuning de BETO binario

**Natalia Dedandi**


Esta notebook presenta el entrenamiento de un clasificador BETO (BERT en español):

https://github.com/dccuchile/beto?tab=readme-ov-file

Nombre del modelo: dccuchile/bert-base-spanish-wwm-uncased

Para el finetuning se trabaja a partir de un dataset etiquetado en castellano argentino elaborado en el marco del proyecto PÍUBAMAS. Más infomación aqui:
https://huggingface.co/piuba-bigdata



In [1]:
!pip install datasets seaborn
! pip install -U accelerate
! pip install -U transformers



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requi

In [2]:
from huggingface_hub import notebook_login
from google.colab import userdata

notebook_login()

In [ ]:
# no lograba bajar el dataset porque
# me fallaba bajar archivos de hugginface y tenia que actualizar esto (JUAN MA me ayudo)
#pip install -U datasets

In [3]:
from datasets import load_dataset
import pandas as pd
from datetime import datetime, timedelta
from tqdm.auto import tqdm
from collections import Counter

(1) Dataset de etiquetado

https://huggingface.co/datasets/piuba-bigdata/contextualized_hate_speech

Este es el dataset original de etiquetado. Contiene los siguientes campos:

ID : identificador

TITLE: título de la noticia

TEXT: texto del tweet del comentario al Tweet original

CONTEXT: tweet original

HATEFUL: 1 hateful, 0 not hateful

BODY: texto de la noticia

CALLS: si se realizó o no un llamado a la acción

WOMEN: discriminación hacia mujeres

LGBTI: discriminación hacia grupos LGBTI

RACISM: discriminación asociada a racismo

CLASS: discriminación vinculada a pobreza, villas

POLITICS: odio o violencia vinculada a las ideas políticas

DISABLED: discriminación a las personas con discapacidad

APPEARENCE: discriminación en base al aspecto físico (incluye especialmente la gordofobia)

CRIMINAL: discriminación hacia personas en el sistema penal o similares



In [4]:
from datasets import load_dataset

ds1 = load_dataset("piuba-bigdata/contextualized_hate_speech")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/36420 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11343 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/9106 [00:00<?, ? examples/s]

In [5]:
ds1

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'text', 'context_tweet', 'HATEFUL', 'body', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL'],
        num_rows: 36420
    })
    test: Dataset({
        features: ['id', 'title', 'text', 'context_tweet', 'HATEFUL', 'body', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL'],
        num_rows: 11343
    })
    dev: Dataset({
        features: ['id', 'title', 'text', 'context_tweet', 'HATEFUL', 'body', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL'],
        num_rows: 9106
    })
})

El dataset 1 contiene 56869 tw etiquetados.
El dataset se dividió en 36420 de train, test:11343 y vaLidacion:9106

In [6]:
ds1_train=ds1["train"]
ds1_train

Dataset({
    features: ['id', 'title', 'text', 'context_tweet', 'HATEFUL', 'body', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL'],
    num_rows: 36420
})

**Para el entrenamiento se usaron los siguientes tutoriales y documentos:**

Uso  de base el siguiente tutorial para hacer el finetuning:
https://huggingface.co/blog/sentiment-analysis-python
y este:
https://huggingface.co/docs/transformers/training


La idea es utilizar la librería tranbsformers de huggingface para hacerle finetuning a un BETO original, es decir la version de BERT pre-entrenada con español de GOOGLE pero que no haya sido reentrenado con ningún otro dataset.
Aqui para usar transformers de hugginface
https://github.com/huggingface/transformers?tab=readme-ov-file



In [7]:
import torch
torch.cuda.is_available()

True

### 1. pre-process data

In [8]:
 ds1['train'][0]

{'id': 343726,
 'title': 'Video: salió de la cárcel por el coronavirus y murió de un tiro el mismo día al festejar su libertad',
 'text': '@usuario Uno menos',
 'context_tweet': 'Video: salió de la cárcel por el coronavirus y murió de un tiro el mismo día al festejar su libertad https://t.co/UrJGRAdEvD',
 'HATEFUL': 1,
 'body': 'Un hombre de 46 años, que cumplía una condena en prisión y fue puesto en libertad por el coronavirus, murió el mismo día que dejó la cárcel baleado por un primo, durante los "festejos" por dejar la prisión.\n\nSari Salem Wardat, de 46 años, recibió un disparo desde atrás en un aparente accidente mientras caminaba hacia miembros de la familia y vecinos que se habían reunido para darle la bienvenida a su casa, en Jordania.\n\nUn video que circuló en las redes sociales \u200bmuestra a Sari llegando a su casa en un auto azul rodeado de tiros de "celebración". El primo se acerca de inmediato a Sari, que sostiene una pistola y la apunta hacia el cielo.\n\nEl detenido

In [9]:
##ME quedo solo con las columnas que necesito para la clasificación binaria
ds_train_small= ds1['train'].select_columns(["text","HATEFUL"])
ds_test_small= ds1['test'].select_columns(["text","HATEFUL"])
ds_val_small= ds1['dev'].select_columns(["text","HATEFUL"])

Nos quedamos con las columnas Text y Hateful y se renombra esta última ya que el modelo requiere que se llame "label"

In [10]:
##aca les coloco el mismo nombre que espera el modelo (me parece cualquiera pero no logre que ande sino)
ds_train_small=ds_train_small.rename_column("HATEFUL", "label")
ds_test_small=ds_test_small.rename_column("HATEFUL", "label")
ds_val_small=ds_val_small.rename_column("HATEFUL", "label")

In [11]:
ds_train_small

Dataset({
    features: ['text', 'label'],
    num_rows: 36420
})

In [12]:

from transformers import BertForMaskedLM, BertTokenizer

Utiizo las librerías de transformers para preparar los datos de entrenamiento

Lo primero es tokenizar en función del modelo seleccionado, en nuestro caso BETO

In [13]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")

tokenizer_config.json:   0%|          | 0.00/310 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/486k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Le aplico la función tokenizer del modelo BETO a mis conjuntos, el de entrenamiento, el de test y el de validación

In [14]:
def preprocess_function(examples):
   return tokenizer(examples["text"], truncation=True)

tokenized_train = ds_train_small.map(preprocess_function, batched=True)
tokenized_test = ds_test_small.map(preprocess_function, batched=True)
tokenized_val = ds_val_small.map(preprocess_function, batched=True)

Map:   0%|          | 0/36420 [00:00<?, ? examples/s]

Map:   0%|          | 0/11343 [00:00<?, ? examples/s]

Map:   0%|          | 0/9106 [00:00<?, ? examples/s]

In [15]:
tokenized_train[0]

{'text': '@usuario Uno menos',
 'label': 1,
 'input_ids': [4, 985, 8563, 1614, 1885, 5],
 'token_type_ids': [0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1]}

La siguiente función (según el ejemplo) transforma los datos de entrenamiento agregandole el padding

Se usará luego en el Trainer

In [16]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## 2 Entreno el modelo.

Voy a entrenar el modelo con el TRAINER de transformers. Instancio el modelo pre-entrenado BETO y aca selecciono 2 labels ya que se hace una clasificación binaria

Modelo: dccuchile/bert-base-spanish-wwm-uncased


In [17]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased", num_labels=2)


pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,roc_auc_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

def compute_metrics(eval_pred):

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    metrics = {}

    metrics['Accuracy'] = accuracy_score(labels, predictions)
    metrics['Precision'] = precision_score(labels, predictions)
    metrics['Recall'] = recall_score(labels, predictions)
    metrics['f1'] = f1_score(labels, predictions)
    metrics['AUC'] = roc_auc_score(labels, predictions)

    return(metrics)



Revision de parametros de TRAINER
https://huggingface.co/docs/transformers/main_classes/trainer

Definicipon de parametros
https://huggingface.co/docs/transformers/v4.41.3/en/main_classes/trainer#transformers.TrainingArguments



In [19]:
from transformers import TrainingArguments, Trainer

repo_name = "beto_hate_speech_ar_bin"

training_args = TrainingArguments(
	 output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=5,
   weight_decay=0.01,
   save_strategy="epoch",
   push_to_hub=True,
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [20]:
# Train pre-trained model
trainer.train()

Step,Training Loss
500,0.359800
1000,0.311500
1500,0.300400
2000,0.293800
2500,0.259600
3000,0.222400
3500,0.220900
4000,0.224100
4500,0.230000
5000,0.163900


TrainOutput(global_step=11385, training_loss=0.17101222671957775, metrics={'train_runtime': 2371.3761, 'train_samples_per_second': 76.791, 'train_steps_per_second': 4.801, 'total_flos': 5422227573295920.0, 'train_loss': 0.17101222671957775, 'epoch': 5.0})

In [21]:
trainer.evaluate()

{'eval_loss': 0.6295255422592163,
 'eval_Accuracy': 0.891563078550648,
 'eval_Precision': 0.6768558951965066,
 'eval_Recall': 0.6037840845854201,
 'eval_f1': 0.6382352941176471,
 'eval_AUC': 0.7747602593469737,
 'eval_runtime': 36.0436,
 'eval_samples_per_second': 314.702,
 'eval_steps_per_second': 19.671,
 'epoch': 5.0}